# Информационный поиск 

Скачиваем классический набор данных — набор текстов об аэронавтике **CRANFIELD**. В случае возникновения проблем с загрузкой — [альтернативная ссылка](https://drive.google.com/file/d/1HDhTQUDJ9ZMKqewikRYQvrFd1nMLll4V/view).

In [1]:
! wget -q http://ir.dcs.gla.ac.uk/resources/test_collections/cran/cran.tar.gz
! tar -xvf cran.tar.gz
! rm cran.tar.gz*

cran.all.1400
cran.qry
cranqrel
cranqrel.readme


Исходный файл **cran.qry** содержит в себе помимо запросов, индексы, от которых нужно избавиться. Берём только запросы, и помещаем их в файл **just.qry**.

In [4]:
! grep -v "^\." cran.qry > just.qry
! head -9 just.qry

what similarity laws must be obeyed when constructing aeroelastic models
of heated high speed aircraft .
what are the structural and aeroelastic problems associated with flight
of high speed aircraft .
what problems of heat conduction in composite slabs have been solved so
far .
can a criterion be developed to show empirically the validity of flow
solutions for chemically reacting gas mixtures based on the simplifying
assumption of instantaneous local chemical equilibrium .


Объединяем многострочные запросы в один.

In [6]:
raw_query_data = [line.strip() for line in open("just.qry", "r").readlines()]
query_data = [""]

for query_part in raw_query_data:
  query_data[-1] += query_part + " "
  if query_part.endswith("."):
    query_data.append("")

query_data[:4] 

['what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft . ',
 'what are the structural and aeroelastic problems associated with flight of high speed aircraft . ',
 'what problems of heat conduction in composite slabs have been solved so far . ',
 'can a criterion be developed to show empirically the validity of flow solutions for chemically reacting gas mixtures based on the simplifying assumption of instantaneous local chemical equilibrium . ']

Составим список интересующих нас запросов.

In [7]:
QUERIES = ['flow behaviour', 'flutter mechanism']

Преобразуем каждый документ в битовую маску (**bitmask**). Битовая маска представляет из себя вектор, в котором на каждой позиции **1**, если в документе есть соответсвующий терм, и **0**, если искомого терма нет.

Для этого понадобится **CountVectorizer()** из библиотеки **scikit-learn**. В рамках поставленной задачи, важно использовать конкретную версию библиотеки.

In [8]:
! pip install -q scikit-learn==0.22.2.post1

In [23]:
from  sklearn.feature_extraction.text import CountVectorizer

encoder = CountVectorizer(binary=True)
encoded_data = encoder.fit_transform(query_data)
encoded_queries = encoder.transform(QUERIES)
list(encoder.vocabulary_)[:8]

['what', 'similarity', 'laws', 'must', 'be', 'obeyed', 'when', 'constructing']

Посмотрим на представление первого предложения.

In [24]:
id2term = {idx: term for term, idx in encoder.vocabulary_.items()}
non_zero_values_ids = encoded_data[0].nonzero()[1]

terms = [id2term[idx] for idx in non_zero_values_ids]
terms

['what',
 'similarity',
 'laws',
 'must',
 'be',
 'obeyed',
 'when',
 'constructing',
 'aeroelastic',
 'models',
 'of',
 'heated',
 'high',
 'speed',
 'aircraft']

Для каждого из запросов в **QUERIES** требуется найти ближайший для него документ из **query_data** по сходству Жаккара (**Jaccard index**). 

Это можно сделать с помощью метода **jaccard_score()** из библиотеки **scikit-learn**.

In [26]:
import numpy as np 
from sklearn.metrics import jaccard_score

def jaccard_sim(vector_a: np.array, vector_b: np.array) -> float:
  return jaccard_score(vector_a, vector_b)
  
assert jaccard_sim(np.array([1, 0, 1, 0, 1]), np.array([0, 1, 1, 1, 1])) == 0.4

В тестовых векторах, поданных на **assert jaccard_sim()** документы представлены как строки в терм-документной матрице (**document-term matrix**). Каждая ячейка вектора отвечает за наличие/отсутствие конкретного элемента (например, слова-терма, когда у нас в словаре всего 5 слов). В первом случае их три ([**1**, 0, **1**, 0, **1**]), во втором — четыре ([0, **1**, **1**, **1**, **1**]). Объединение (**Union**) — все пять возможных элементов. Пересечение (**Intersection**) — два. Следовательно Сходство Жаккара = intersection(vector_a, vector_b)/union(vector_a, vector_b) = 2/5 = **0.4**.

Вычислим для каждого запроса **QUERIES** самые близкие документы **query_data**.

In [27]:
for q_id, query in enumerate(encoded_queries):
  query = query.todense().A1
  docs = [doc.todense().A1 for doc in encoded_data]
  id2doc2similarity = [(doc_id, doc, jaccard_sim(query, doc)) for doc_id, doc in enumerate(docs)]
  closest = sorted(id2doc2similarity, key=lambda x: x[2], reverse=True)
  
  print("Q: %s\nFOUND:" % QUERIES[q_id])
  for closest_id, _, sim in closest[:3]:
    print("    %d\t%.2f\t%s" %(closest_id, sim, query_data[closest_id]))



Q: flow behaviour
FOUND:
    5	0.14	what theoretical and experimental guides do we have as to turbulent couette flow behaviour . 
    29	0.12	papers on flow visualization on slender conical wings . 
    8	0.11	papers on internal /slip flow/ heat transfer studies . 
Q: flutter mechanism
FOUND:
    185	0.17	experimental studies on panel flutter . 
    56	0.15	what are the significant steady and non-steady flow characteristics which affect the flutter mechanism . 
    12	0.10	what is the basic mechanism of the transonic aileron buzz . 


А теперь сделаем всё то же самое, но уже с помощью **TF-IDF** и Косинусного Расстояния (**Cosine Distance**).

In [30]:
from  sklearn.feature_extraction.text import TfidfVectorizer

tfidf_encoder = TfidfVectorizer()
tfidf_encoded_data = tfidf_encoder.fit_transform(query_data)
tfidf_encoded_queries = tfidf_encoder.transform(QUERIES)

list(tfidf_encoder.vocabulary_)[:8]

['what', 'similarity', 'laws', 'must', 'be', 'obeyed', 'when', 'constructing']

Найдём Косинусное Расстояние (**Cosine Distance**) между двумя векторами. Для этого воспользуемся методом **cosine()** библиотеки **scipy**.

In [31]:
import numpy as np 
import scipy.spatial.distance as ds

def cosine_distance(vector_a: np.array, vector_b: np.array) -> float:
  return ds.cosine(vector_a, vector_b)

assert cosine_distance(np.array([1, 0, 1, 1, 1]), np.array([0, 0, 1, 0, 0])) == 0.5


Теперь вычислим ближайшие по **Косинусному Расстоянию** между векторными представлениями документов и запросов

In [32]:
for q_id, query in enumerate(tfidf_encoded_queries):
  query = query.todense().A1
  docs = [doc.todense().A1 for doc in tfidf_encoded_data]
  id2doc2similarity = [(doc_id, doc, cosine_distance(query, doc)) for doc_id, doc in enumerate(docs)]
  closest = sorted(id2doc2similarity, key=lambda x: x[2], reverse=False)
  
  print("Q: %s\nFOUND:" % QUERIES[q_id])
  
  for closest_id, _, sim in closest[:3]:
    print("    %d\t%.2f\t%s" %(closest_id, sim, query_data[closest_id]))

Q: flow behaviour
FOUND:
    5	0.65	what theoretical and experimental guides do we have as to turbulent couette flow behaviour . 
    149	0.75	has anyone developed an analysis which accurately establishes the large deflection behaviour of conical shells . 
    146	0.77	does a membrane theory exist by which the behaviour of pressurized membrane cylinders in bending can be predicted . 
Q: flutter mechanism
FOUND:
    56	0.63	what are the significant steady and non-steady flow characteristics which affect the flutter mechanism . 
    12	0.68	what is the basic mechanism of the transonic aileron buzz . 
    185	0.72	experimental studies on panel flutter . 


/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
